In [218]:
import pandas as pd
import numpy as np
import json
import time
import random
import pickle

from pytrends.request import TrendReq
pytrends = TrendReq(hl='en-US', tz=360)


## https passed proxies: 	https://20.99.187.69:8443, https://34.16.139.133:8585, https://35.245.181.60:8585

In [186]:
json_data = [] # your list with json objects (dicts)

with open('counterfact/counterfact.json') as json_file:
   json_data = json.load(json_file)

In [187]:
subjects = list(set([x['requested_rewrite']['subject'] for x in json_data]))
relations = list(set([x['requested_rewrite']['relation_id'] for x in json_data]))
print(len(relations))

34


In [188]:
# for each relation, first find most popular thing
# 20k things / 5 per query = 4k queries

# then, score all others gainst most popular, 4 at a time
# 20k things / 4 per query = 5k queries

def find_most_popular_batch(kw_list):
    pytrends.build_payload(kw_list = kw_list)
    res = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
    out = res.loc["United States"].sort_values(ascending=False).index[0]

    return(out)

def find_most_popular(x):
    i = 1
    best = x[0]
    while i < len(x):
        l = x[i:i+4]
        kw_list = [best] + l
        best = find_most_popular_batch(kw_list)
        i+= 4
        time.sleep(1.1)

    return(best)



In [199]:


def popularity_score_batch(input_list, reference):
    pytrends.build_payload(kw_list = input_list + [reference])
    res = pytrends.interest_by_region(resolution='COUNTRY', inc_low_vol=False, inc_geo_code=False)
    out = res.loc["United States"]


    return(out)


def popularity_scores(x, reference):
    i = 0
    out = pd.Series()
    x.remove(reference)
    while i < len(x):
        l = x[i:i+4]
        scores = popularity_score_batch(l, reference)
        out = pd.concat([out, scores[0:4]])
        # out = out.append(z2, )
        i+= 4
        time.sleep(1.1)

    return(out)

In [121]:
s_p1303 = [x['requested_rewrite']['subject'] for x in json_data if x['requested_rewrite']['relation_id']=="P1303"]

most_pop = find_most_popular(s_p1303[0:100])
pop_scores = popularity_scores(s_p1303[0:100], most_pop)


In [134]:
# pop_scores.sort_values(ascending=False).head(10)
pop_scores[pop_scores > 0].sort_values(ascending=False)

Britney Spears    96
Britney Spears    50
The Edge          45
Elvis Presley     39
George Martin     10
Sheryl Crow        8
Cat Stevens        6
Tom Waits          5
Brian May          5
Chris Daughtry     4
John Fogerty       4
Julian Lennon      2
David Gates        1
Chris Thile        1
Peter Head         1
Nita Strauss       1
Franz Liszt        1
Name: United States, dtype: int64

In [227]:
def get_subj_by_rid(relation_id):
    subjs = [x['requested_rewrite']['subject'] for x in json_data if x['requested_rewrite']['relation_id']==relation_id]
    return(list(set(subjs)))

In [219]:
## for each relation type, find the popularity scores!
res_dict = dict()

In [228]:
for r in relations[4:]:
    print("scoring popularity for relation:", r)
    subjs = get_subj_by_rid(r)
    # subjs = random.choices(get_subj_by_rid(r), k=20)
    ref = find_most_popular(subjs)
    time.sleep(1.1)
    scores = popularity_scores(subjs, ref)
    res_dict[r] = {"ref": ref, "scores": scores}

scoring popularity for relation: P27


TooManyRequestsError: The request failed: Google returned a response with code 429

In [221]:
# Store data (serialize)
with open('counterfact/pop-results.pickle', 'wb') as handle:
    pickle.dump(res_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)


In [204]:
# # Load data (deserialize)
# with open('counterfact/pop-test.pickle', 'rb') as handle:
#     unserialized_data = pickle.load(handle)

# unserialized_data["P1303"]["scores"]

In [226]:
res_dict["P1303"]["scores"].sort_values(ascending=False).head(20)

Britney Spears     45
The Edge           43
Beck               42
Elvis Presley      36
Aretha Franklin    28
Garth Brooks       28
Paul McCartney     22
Jared Leto         16
Jared Leto         16
Jimi Hendrix       14
Eric Clapton       14
Eric Clapton       14
Rob Zombie         13
Ray Charles        10
George Martin       8
Sheryl Crow         7
Leonard Cohen       6
Joe Walsh           6
Nat King Cole       5
Cat Stevens         5
dtype: int64